# Ajustes Interativos de Imagem

Controle Matiz (Hue), Saturação e Contraste de imagens de microplásticos com sliders interativos.

## Funções OpenCV Utilizadas:

### Conversões de Espaço de Cor:
- **`cv2.COLOR_BGR2RGB`**: Converte de BGR (padrão OpenCV) para RGB (padrão visualização)
- **`cv2.COLOR_RGB2HSV`**: Converte RGB para HSV (Hue-Saturation-Value)
- **`cv2.COLOR_HSV2RGB`**: Converte HSV de volta para RGB

### Espaço de Cor HSV:
- **H (Hue/Matiz)**: Cor base (0-179° no OpenCV, 0-360° tradicionalmente)
- **S (Saturation/Saturação)**: Intensidade da cor (0=cinza, 255=cor pura)
- **V (Value/Valor)**: Brilho da cor (0=preto, 255=brilho máximo)

### Vantagens do HSV para Ajustes:
- **Separação clara** entre cor (H), intensidade (S) e brilho (V)
- **Ajustes independentes** de cada componente
- **Mais intuitivo** para manipulação de cores do que RGB

### Operações Matemáticas:
- **Módulo (%)**: Para matiz circular (0-179°)
- **Multiplicação**: Para saturação e contraste
- **Clipping**: Para manter valores dentro dos limites válidos

In [1]:
# Importação das bibliotecas necessárias
import cv2  # OpenCV para processamento de imagens
import numpy as np  # NumPy para operações matemáticas com arrays
import matplotlib.pyplot as plt  # Matplotlib para visualização de gráficos
from ipywidgets import interact, FloatSlider, Dropdown  # Widgets interativos do Jupyter
import os  # Biblioteca para operações do sistema operacional
from IPython.display import display  # Para exibição no Jupyter Notebook

In [2]:
# Carregamento das imagens disponíveis no diretório
data_dir = '../data'  # Diretório onde estão armazenadas as imagens
supported_formats = ('.jpg', '.jpeg', '.png', '.tiff', '.tif')  # Formatos de imagem suportados

image_files = []  # Lista para armazenar nomes dos arquivos de imagem
if os.path.exists(data_dir):  # Verifica se o diretório existe
    # Filtra arquivos que terminam com extensões suportadas (case-insensitive)
    image_files = [f for f in os.listdir(data_dir) 
                   if f.lower().endswith(supported_formats)]

if not image_files:
    print("Nenhuma imagem encontrada no diretório de dados")
else:
    print(f"Encontradas {len(image_files)} imagens: {image_files}")

Encontradas 26 imagens: ['P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'PCB1.1_LK_Q5.jpg', 'P1A_T1_LD_Q5_100X.jpg', 'CNN_P1_L1_T1_M1_LG_Q8_150x.jpg', 'PCB1.1_LK_Q7.jpg', 'NIVEL 2 (BR1_1CAMP_LO_Q6_125X).jpg', 'PCB1.1_LM_.jpg', 'NIVEL 5 (P1D_T1_LI_Q11_63X).jpg', 'NIVEL 4 (P1D_T2_LJ_Q10_150X).jpg', 'P1A_T1_LC_Q6_100X.jpg', 'CNN_P1_L1_T1_M1_LH_Q3_63x.jpg', 'CNN_P1_L1_T1_M1_LI_Q2_50x.jpg', 'P1A_T1_LG_Q8_150X (1).jpg', 'PCB1.1.jpg', 'NIVEL 1 (BR3_1CAMP_LC_Q6_100X).jpg', 'NIVEL 3 (P1D_T3_LH_Q8_100X (2)).jpg', 'PCB1.1LB_Q9.jpg', 'CNN_P1_L1_T1_M1_LG_Q5_80x.jpg', 'PCB1.1_LK_Q10.jpg', 'CNN_P1_L1_T1_M1_LH_Q9_F1_150x.jpg', 'PCB1.1_LB_Q6.jpg', 'CNN_P1_L1_T1_M1_LC_Q9_100x.jpg', 'CNN_P1_L1_T1_M1_LG_Q11_100x.jpg', 'CNN_P1_L1_T1_M1_LI_Q6_125x.jpg', 'P1A_T1_LI_Q11_40X.jpg']


In [3]:
def adjust_image(image_path, hue=0, saturation=1.0, contrast=1.0):
    """
    Ajusta matiz, saturação e contraste de uma imagem
    
    Parâmetros:
    - image_path: caminho para o arquivo de imagem
    - hue: ajuste de matiz em graus (-180 a +180)
    - saturation: multiplicador de saturação (0.0 a 3.0)
    - contrast: multiplicador de contraste (0.0 a 3.0)
    
    Retorna:
    - adjusted: imagem ajustada em formato RGB
    """
    # Carrega a imagem (OpenCV usa formato BGR por padrão)
    img = cv2.imread(image_path)
    
    # Converte de BGR para RGB (formato padrão para visualização)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Converte para espaço de cor HSV para ajustes de matiz e saturação
    # HSV separa cor (H), saturação (S) e brilho (V)
    # .astype(np.float32) converte para float para cálculos precisos
    hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV).astype(np.float32)
    
    # Ajusta matiz (canal H - índice 0)
    # Soma o valor de ajuste e usa módulo 180 para manter no range válido
    # OpenCV usa range 0-179 para matiz (ao invés de 0-359 tradicional)
    hsv[:,:,0] = (hsv[:,:,0] + hue) % 180
    
    # Ajusta saturação (canal S - índice 1)
    # Multiplica pela saturação e usa clip para manter entre 0-255
    # np.clip garante que valores não ultrapassem os limites
    hsv[:,:,1] = np.clip(hsv[:,:,1] * saturation, 0, 255)
    
    # Converte de volta para RGB
    # .astype(np.uint8) converte de volta para inteiros de 8 bits
    adjusted = cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2RGB)
    
    # Ajusta contraste multiplicando todos os canais RGB
    # np.clip garante que valores fiquem entre 0-255
    adjusted = np.clip(adjusted * contrast, 0, 255).astype(np.uint8)
    
    return adjusted

In [4]:
def interactive_adjustment(image_name, hue, saturation, contrast):
    """
    Função interativa para ajuste de imagem com visualização em tempo real
    
    Parâmetros:
    - image_name: nome do arquivo de imagem selecionado
    - hue: valor de ajuste de matiz (-180 a +180)
    - saturation: multiplicador de saturação (0.0 a 3.0)
    - contrast: multiplicador de contraste (0.0 a 3.0)
    """
    if not image_files:
        print("Nenhuma imagem disponível")
        return
    
    # Constrói o caminho completo para a imagem selecionada
    image_path = os.path.join(data_dir, image_name)
    
    # Aplica os ajustes na imagem
    adjusted = adjust_image(image_path, hue, saturation, contrast)
    
    # Cria figura com tamanho específico (10x5 polegadas)
    plt.figure(figsize=(10, 5))
    
    # Carrega e exibe imagem original (lado esquerdo)
    original = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    plt.subplot(1, 2, 1)  # 1 linha, 2 colunas, posição 1
    plt.imshow(original)  # Exibe imagem original
    plt.title('Original')
    plt.axis('off')  # Remove eixos para melhor visualização
    
    # Exibe imagem ajustada (lado direito)
    plt.subplot(1, 2, 2)  # 1 linha, 2 colunas, posição 2
    plt.imshow(adjusted)  # Exibe imagem com ajustes aplicados
    # Título mostra valores atuais dos parâmetros
    plt.title(f'M:{hue:.0f}° S:{saturation:.1f}x C:{contrast:.1f}x')
    plt.axis('off')
    
    # Ajusta layout para evitar sobreposição de elementos
    plt.tight_layout()
    plt.show()  # Exibe a figura

In [6]:
# Widget interativo para ajustes de imagem
if image_files:
    interact(interactive_adjustment,
             # Dropdown para seleção da imagem
             image_name=Dropdown(options=image_files, description='Imagem:'),
             
             # Slider para ajuste de matiz (-180° a +180°)
             # step=1 permite ajustes de 1 grau
             hue=FloatSlider(min=-180, max=180, step=1, value=0, 
                           description='Matiz (°):'),
             
             # Slider para saturação (0x a 3x)
             # 0 = imagem em escala de cinza, 1 = original, >1 = mais saturada
             saturation=FloatSlider(min=0, max=3, step=0.1, value=1.0, 
                                  description='Saturação (x):'),
             
             # Slider para contraste (0x a 3x)
             # 0 = imagem preta, 1 = original, >1 = mais contraste
             contrast=FloatSlider(min=0, max=3, step=0.1, value=1.0, 
                                description='Contraste (x):'))
else:
    print("Adicione imagens ao diretório ../data/ para usar os ajustes interativos")

interactive(children=(Dropdown(description='Imagem:', options=('P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'P…